In [2]:
import pandas as pd

# Load the dataset
file_path = 'train.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 269.6/269.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 97.9/97.9 kB 2.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import nltk
from nltk.corpus import stopwords

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baska\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import string

# Load the dataset
data = pd.read_csv('test.csv')

# Define stopwords (as you have already done)
stopwords = set([
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', 
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
    'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
    'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 
    'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 
    'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
    'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 
    "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
])
stopwords_list = list(stopwords)

# # Vectorize the questions using TF-IDF
# vectorizer = TfidfVectorizer(stop_words=stopwords_list)
# X1 = vectorizer.fit_transform(data['question1'])
# X2 = vectorizer.transform(data['question2'])
# Define preprocessing function (as you have already done)
def preprocess(text):
    if pd.isnull(text):
        return ""
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Split into words and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)
# Apply preprocessing to the questions
data['question1'] = data['question1'].apply(preprocess)
data['question2'] = data['question2'].apply(preprocess)

# Vectorize the questions using TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_list)
X1 = vectorizer.fit_transform(data['question1'])
X2 = vectorizer.transform(data['question2'])

# Combine the vectors for clustering
X_combined = pd.concat([pd.DataFrame(X1.toarray()), pd.DataFrame(X2.toarray())], ignore_index=True)

# Apply KMeans clustering
num_clusters = 5  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X_combined)

# Get the cluster labels for each question
clusters = kmeans.labels_

# Add the cluster labels back to the original questions
data['cluster_q1'] = clusters[:len(data)]
data['cluster_q2'] = clusters[len(data):]

# Determine if the questions in each pair are in the same cluster
data['same_cluster'] = data['cluster_q1'] == data['cluster_q2']

# Display the results
print(data[['question1', 'question2', 'same_cluster']].head())


                                question1  \
0          surface pro 4 compare ipad pro   
1  hair transplant age 24 much would cost   
2            best way send money china us   
3                        food emulsifiers   
4               aberystwyth start reading   

                                           question2  same_cluster  
0  microsoft choose core m3 core i3 home surface ...          True  
1                  much cost hair transplant require          True  
2                                   send money china          True  
3                                        foods fibre          True  
4                                      start reading          True  


C:\Users\baska\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [24]:
data.to_csv('quora_data_with_clusters.csv', index=False)